In [15]:
# !pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)

2.4.1


In [16]:
from __future__ import division, print_function, unicode_literals
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os, os.path
import cv2
import random

In [17]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [18]:
np.random.seed(42)

In [25]:
def image_noaugmentation(imagesArray,imagesLabels,path):
    full_path = path
    ImagesPaths = [x for x in os.listdir(full_path)]
    
    count = 0
    for i in ImagesPaths:
        img = cv2.imread(full_path+i)
        #img = cv2.resize(img,(img_width,img_height))
        imagesArray[count]= img
        imagesLabels[count]=np.array(0)
        count+=1
        
    full_path = path
    ImagesPaths = [x for x in os.listdir(full_path)]
    
    for i in ImagesPaths:
        img = cv2.imread(full_path+i)
        #img = cv2.resize(img,(img_width,img_height))
        imagesArray[count]= img
        imagesLabels[count]=np.array(1)
        count+=1

In [26]:
from tensorflow.keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives)
        return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives)
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [27]:
K.epsilon()

1e-07

In [28]:
img_width, img_height, img_num_channels = 50, 50, 3
#search about face Pixels


input_shape = (img_width, img_height, img_num_channels)
pathTrain = 'D:\\Graduation Project\\dataset\\train'
imagesArrayTrain = np.zeros((2102,img_width,img_height,img_num_channels),dtype = 'float32')
imagesLabelsTrain = np.zeros((2102,1),dtype = 'uint8')

In [29]:
pathTest = 'D:\\Graduation Project\\dataset\\test'
imagesArrayTest = np.zeros((196,img_width,img_height,img_num_channels),dtype = 'float32')
imagesLabelsTest = np.zeros((196,1),dtype = 'uint8')

In [30]:
image_noaugmentation(imagesArrayTrain,imagesLabelsTrain,pathTrain)
imagesArrayTrain = imagesArrayTrain / 255
shuffler = np.random.permutation(len(imagesArrayTrain))
imagesArrayTrain_shuffled = imagesArrayTrain[shuffler]
imagesLabelsTrain_shuffled = imagesLabelsTrain[shuffler]

In [31]:
image_noaugmentation(imagesArrayTest,imagesLabelsTest,pathTest)
imagesArrayTest = imagesArrayTest / 255
shuffler = np.random.permutation(len(imagesArrayTest))
X_test = imagesArrayTest[shuffler]
y_test = imagesLabelsTest[shuffler]

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(imagesArrayTrain_shuffled, imagesLabelsTrain_shuffled,stratify=imagesLabelsTrain_shuffled, test_size=0.2)

In [35]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
X = tf.placeholder(shape=[None, 50, 50, 3], dtype=tf.float32, name="X")

Instructions for updating:
non-resource variables are not supported in the long term


In [36]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 17 * 17  # 1152 primary capsules
caps1_n_dims = 8

In [37]:
conv1_params = {
    "filters": 256,
    "kernel_size": 9,
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}

In [38]:
conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)

C:\Users\Rawan Khaled\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
C:\Users\Rawan Khaled\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [39]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")

In [40]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [41]:
caps1_output = squash(caps1_raw, name="caps1_output")


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [42]:
caps2_n_caps = 10
caps2_n_dims = 16

In [43]:
init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

In [44]:
batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

In [45]:
caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

In [46]:
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")

In [47]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")

In [48]:
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")


Instructions for updating:
dim is deprecated, use axis instead


In [49]:
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")

In [50]:
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")

In [51]:
caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

In [52]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

In [53]:
raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

In [54]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")


In [55]:
caps2_output = caps2_output_round_2


In [56]:
def condition(input, counter):
    return tf.less(counter, 100)

def loop_body(input, counter):
    output = tf.add(input, tf.square(counter))
    return output, tf.add(counter, 1)

with tf.name_scope("compute_sum_of_squares"):
    counter = tf.constant(1)
    sum_of_squares = tf.constant(0)

    result = tf.while_loop(condition, loop_body, [sum_of_squares, counter])
    

with tf.Session() as sess:
    print(sess.run(result))


(328350, 100)


In [57]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)


In [58]:
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")


In [59]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")


In [60]:
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
y_pred

<tf.Tensor 'y_pred:0' shape=(?,) dtype=int64>

In [61]:
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")
y

<tf.Tensor 'y:0' shape=(?,) dtype=int64>

In [62]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

In [63]:
T = tf.one_hot(y, depth=caps2_n_caps, name="T")


In [64]:
with tf.Session():
    print(T.eval(feed_dict={y: np.array([0, 1, 2, 3, 9])}))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [65]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

In [66]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 10),
                           name="present_error")

In [67]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 10),
                          name="absent_error")

In [68]:
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

In [69]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")


In [70]:
mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")

In [71]:
reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")

In [72]:
reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

In [73]:
reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")

In [74]:
caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

In [75]:
decoder_input = tf.reshape(caps2_output_masked,
                           [-1, caps2_n_caps * caps2_n_dims],
                           name="decoder_input")

In [76]:
n_hidden1 = 512
n_hidden2 = 1024
n_output = 50 * 50 * 3

In [77]:
with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                              activation=tf.nn.relu,
                              name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu,
                              name="hidden2")
    decoder_output = tf.layers.dense(hidden2, n_output,
                                     activation=tf.nn.sigmoid,
                                     name="decoder_output")

C:\Users\Rawan Khaled\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '


In [78]:
X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                    name="reconstruction_loss")

In [79]:
alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")

In [80]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [81]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

In [82]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [83]:
y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()

In [84]:
test_prec = precision_m(y,y_pred)
test_recall =recall_m(y,y_pred)
test_f1 = f1_m(y,y_pred)

In [85]:
n_epochs = 10
batch_size = 20
restore_checkpoint = True

trainLen = len(X_train)
valLen = len(X_val)


n_iterations_per_epoch = trainLen // batch_size
n_iterations_validation = valLen // batch_size
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

with tf.Session(config=config) as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()

    for epoch in range(n_epochs):
        start = 0
        end = batch_size
        for iteration in range(1, n_iterations_per_epoch + 1):
            X_batch, y_batch = X_train[start:end],y_train[start:end]
            # Run the training operation and measure the loss:
            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 50, 50, 3]),
                           y: y_batch,
                           mask_with_labels: True})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                      iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train),
                  end="")
            start = start+batch_size
            end = end+batch_size

        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        prec_vals = []
        recall_vals = []
        f1_vals = []
        start = 0
        end = batch_size
        for iteration in range(1, n_iterations_validation + 1):
            X_batch, y_batch = X_val[start:end],y_val[start:end]
            loss_val, acc_val,prec_test,recall_test,f1_m_test = sess.run(
                    [loss, accuracy,test_prec,test_recall,test_f1],
                    feed_dict={X: X_batch.reshape([-1,50, 50, 3]),
                               y: y_batch})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            prec_vals.append(prec_test)
            recall_vals.append(recall_test)
            f1_vals.append(f1_m_test)
            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                  end=" " * 10)
            start = start+batch_size
            end = end+batch_size
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        prec_val = np.mean(prec_vals)
        recall_val = np.mean(recall_vals)
        f1_val = np.mean(f1_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f} Precision:{}% Recall:{}% F1_measure:{}% {}".format(
            epoch + 1, acc_val * 100, loss_val, prec_val*100, recall_val*100, f1_val*100,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val
        

Instructions for updating:
Use standard file APIs to check for files with this prefix.
Epoch: 1  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 2  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 3  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 4  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 5  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 6  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 7  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 8  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 9  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
Epoch: 10  Val accuracy: 99.7619%  Loss: nan Precision:nan% Recall:nan% F1_measure:nan% 
